In [ ]:
%load_ext autoreload
%autoreload 2
from functions.seq_autoencoder import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import os
import argparse

import sys
sys.path.insert(1, '../NRI') # include NRI files in the $PATH
from modules import *
from utils import *

# Define folder & load files

In [ ]:
! ls ../NRI/logs

In [ ]:
! ls -latrh ../NRI/logs/dance_53joints_rnndecoder_sparse_k6/

In [ ]:
folder = "../NRI/logs/dance_53joints_rnndecoder_sparse_k6/exp2020-07-31T20:29:06.738927"

In [ ]:
meta_file = os.path.join(folder, 'metadata.pkl')
args = pickle.load(open(meta_file, "rb" ))['args']

### Load test file predictions, ground truth, and edge type labels

With sampling:

In [ ]:
predicted_outputs = pickle.load(open(os.path.join(folder,"predicted_outputs.txt"), "rb"))
actual_outputs = pickle.load(open(os.path.join(folder,"actual_outputs.txt"), "rb"))
edge_types = pickle.load(open(os.path.join(folder,"edges.txt"), "rb"))

In [ ]:
batch_number = 2
actual = actual_outputs[0][batch_number, :, :, :3]
pred = predicted_outputs[0][batch_number, :, :, :3]
edge_types = edge_types[0][batch_number, :, :]

actual = np.transpose(actual, [1, 2, 0])
pred = np.transpose(pred, [1, 2, 0])

actual = actual.reshape((999,53,3))
pred = pred.reshape((999,53,3))

With argmax:

In [ ]:
predicted_outputs_argmax = pickle.load(open(os.path.join(folder,"predicted_outputs_argmax.txt"), "rb"))
actual_outputs_argmax = pickle.load(open(os.path.join(folder,"actual_outputs_argmax.txt"), "rb"))
edge_types_argmax = pickle.load(open(os.path.join(folder,"edges_argmax.txt"), "rb"))
logits = pickle.load(open(os.path.join(folder,"logits.txt"), "rb"))

In [ ]:
batch_number = 10
actual = actual_outputs_argmax[0][batch_number, :, :, :3]
pred = predicted_outputs_argmax[0][batch_number, :, :, :3]
edge_types = edge_types_argmax[0][batch_number, :, :]
logits = logits[0][batch_number, :, :]

actual = np.transpose(actual, [1, 2, 0])
pred = np.transpose(pred, [1, 2, 0])

actual = actual.reshape((999,53,3))
pred = pred.reshape((999,53,3))

In [ ]:
# normalize to turn the logits into opacities
edge_opacities = (logits-np.min(logits))/(np.max(logits)-np.min(logits)) 

In [ ]:
# percentage of the edges that belong to each class
percentages = edge_types.sum(axis=0)/edge_types.shape[0]

for i in range(len(percentages)):
    print("Edge type {}: {:.1f}%".format(i, percentages[i]*100))

In [ ]:
for i in range(6):
    plt.hist(edge_opacities[:,i], bins=np.linspace(0,1,50), label=i, histtype="stepfilled", linewidth=2)
plt.legend()

In [ ]:
np.percentile(edge_opacities[:,2],98)

In [ ]:
np.percentile(edge_opacities[:,edge_class],99.9)

In [ ]:
edge_class = 3

start_index = 0 #15*49
timesteps = 49
animation = animate_stick(actual[start_index:start_index+timesteps,:,:], 
#                           ghost=pred[start_index:start_index+timesteps,:,:], 
#                           ghost_shift=0.4,
                          cloud=True,
                          cloud_alpha=0.03, # default=0.03 or 0.1
                          edge_types=edge_types,
                          edge_opacities=edge_opacities, 
                          threshold=np.percentile(edge_opacities[:,edge_class],99.4), # only plot the edges above this threshold with opacities given by scaled argmax
                          edge_class = edge_class,
                          skeleton = True,
                          ax_lims = (-0.5,0.5),
                          figsize=(10,8), 
                          cmap='Greys',
                         )
HTML(animation.to_html5_video())

# Other plots

In [ ]:
start_index = 0
timesteps = 100
fig, ax = plt.subplots(figsize=(8,6))
for joint in range(1):
    ax.plot(actual[start_index:start_index+timesteps,joint,0],actual[start_index:start_index+timesteps,joint,1], 'o--', label="True Joint "+str(joint))
    ax.plot(pred[start_index:start_index+timesteps,joint,0],pred[start_index:start_index+timesteps,joint,1], 'o--', label="Predicted Joint "+str(joint))
ax.set_xlabel("X", fontsize=16)
ax.set_ylabel("Y", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
# ax.set_xlim(-1,1)
# ax.set_ylim(-1,1)
ax.legend(fontsize=16)

In [ ]:
mse_val_31 = [0.0005380539,0.0002545162,0.0002720219,0.0002327177,0.0002369935,0.0002251789,0.0002252262,0.0002203526,0.0002198812,0.0002173202,0.0002166492,0.0002143005,0.0002126976,0.0002115302,0.0002100576,0.0002117148,0.0002137985,0.0002112656,0.0002105504,0.0002097648,0.0002081067,0.0002078729,0.0002088487,0.0002070423,0.0002034100,0.0002038339,0.0002040366,0.0002012061,0.0002000649,0.0001972399,0.0001957860,0.0001958387,0.0001947850,0.0001950850,0.0001947366,0.0001938091,0.0001950343,0.0001947872,0.0001893951,0.0001854542,0.0001849767,0.0001846175,0.0001841435,0.0001845031,0.0001805753,0.0001805988,0.0001809108,0.0001789665,0.0001797675,0.0001769623,0.0001758305,0.0001743786,0.0001731374,0.0001717589,0.0001709805,0.0001702003,0.0001694033,0.0001676418,0.0001708879,0.0001664068,0.0001679793,0.0001662830,0.0001653230,0.0001632752,0.0001634039,0.0001620605,0.0001628637,0.0001612367,0.0001601394,0.0001605294,0.0001620230,0.0001614228,0.0001601921,0.0001587805,0.0001567188,0.0001578624,0.0001570027,0.0001560244,0.0001540982,0.0001545769,0.0001544053,0.0001535158,0.0001540448,0.0001520058,0.0001515924,0.0001516773,0.0001504958,0.0001504630,0.0001503649,0.0001484780,0.0001491130,0.0001485511,0.0001470489,0.0001468935,0.0001453965,0.0001459502,0.0001475472,0.0001474322,0.0001461496,0.0001449367]
mse_val_53 = [0.0003319341,0.0002407579,0.0001978217,0.0001807092,0.0001761422,0.0001762023,0.0001690495,0.0001672494,0.0001659541,0.0001640345,0.0001620488,0.0001620720,0.0001643264,0.0001700758,0.0001762014,0.0001723801,0.0001702851,0.0001764964,0.0001640909,0.0001674481,0.0001652822,0.0001731986,0.0001729803,0.0001682613,0.0001643399,0.0001644315,0.0001635181,0.0001606397,0.0001605609,0.0001724272,0.0001706513,0.0001681733,0.0001636652,0.0001542961,0.0001523678,0.0001497547,0.0001499650,0.0001461304,0.0001477886,0.0001431693,0.0001436184,0.0001417179,0.0001401840,0.0001380969,0.0001414188,0.0001489541,0.0001378575,0.0001364077,0.0001385690,0.0001342427,0.0001340273,0.0001370628,0.0001317071,0.0001317224,0.0001285705,0.0001293033,0.0001294561,0.0001278509,0.0001264333,0.0001280046,0.0001256976,0.0001291989,0.0001254917,0.0001247280,0.0001243784,0.0001220349,0.0001215896,0.0001196151,0.0001198853,0.0001263810,0.0001192785,0.0001228473,0.0001205917,0.0001201656,0.0001168592,0.0001170775,0.0001152546,0.0001147493,0.0001147641,0.0001148067,0.0001132866,0.0001125762,0.0001151791,0.0001156654,0.0001125780,0.0001129397,0.0001123863,0.0001145972,0.0001108145,0.0001097617,0.0001092943,0.0001081585,0.0001092159,0.0001079638,0.0001066594,0.0001071093,0.0001062212,0.0001083658,0.0001086592,0.0001058198]
mse_val_53_pred0 = [0.0002916463,0.0002257201,0.0001934857,0.0001737289,0.0001659179,0.0001671346,0.0001618562,0.0001576361,0.0001542611,0.0001511301,0.0001480298,0.0001450186,0.0001424480,0.0001396098,0.0001427588,0.0001366035,0.0001361172,0.0001329779,0.0001283397,0.0001243424,0.0001241281,0.0001221446,0.0001235387,0.0001275972,0.0001175373,0.0001167690,0.0001175446,0.0001138587,0.0001107952,0.0001163278,0.0001110801,0.0001101223,0.0001084899,0.0001088448,0.0001034001,0.0001039315,0.0001036894,0.0001418207,0.0001100666,0.0001088340,0.0001011293,0.0000997004,0.0000997282,0.0000999370,0.0000985010,0.0000970325,0.0000989251,0.0001157353,0.0001041943,0.0000973772,0.0000959852,0.0000953575,0.0000936873,0.0000938873,0.0000924618,0.0000918216,0.0000935097,0.0001141488,0.0000955536,0.0000957749,0.0000921481,0.0000890592,0.0000897373,0.0000884137,0.0000870536,0.0000867153,0.0000862568,0.0000858130,0.0000899170,0.0001215506,0.0000973756,0.0000859231,0.0000870565,0.0000860288,0.0000825410,0.0000827993,0.0000817308,0.0000807608,0.0000802745,0.0000793182,0.0000787936,0.0000833381,0.0001232066,0.0000971155,0.0000776603,0.0000822653,0.0000785578,0.0000757666,0.0000763259,0.0000748265,0.0000739248,0.0000753339,0.0000838695,0.0000720847,0.0000724253,0.0000716465,0.0000712332,0.0000706910,0.0000706619,0.0000702678]
mse_val_53_pred45 = [0.0010585249,0.0005130282,0.0003761802,0.0002834579,0.0002935287,0.0002391723,0.0002098410,0.0002207255,0.0002079938,0.0001970553,0.0001965527,0.0001937480,0.0001904944,0.0001891827,0.0001879801,0.0001867464,0.0001862566,0.0001851096,0.0001843101,0.0001839120,0.0001828077,0.0001821922,0.0001812789,0.0001805127,0.0001799234,0.0001793888,0.0001787367,0.0001782920,0.0001776933,0.0001771595,0.0001765995,0.0001762180,0.0001758815,0.0001755280,0.0001752425,0.0001747227,0.0001742240,0.0001741879,0.0001735637,0.0001732304,0.0001727253,0.0001723513,0.0001719021,0.0001714883,0.0001711844,0.0001706581,0.0001702720,0.0001699703,0.0001696720,0.0001692688,0.0001690084,0.0001687691,0.0001684537,0.0001680404,0.0001677989,0.0001675765,0.0001673441,0.0001670581,0.0001668098,0.0001665871,0.0001664216,0.0001662568,0.0001659940,0.0001658073,0.0001656917,0.0001654395,0.0001652718,0.0001651123,0.0001650113,0.0001648165,0.0001645865,0.0001644920,0.0001643535,0.0001642707,0.0001640353,0.0001638883,0.0001639180,0.0001637940,0.0001636475,0.0001634188,0.0001633208,0.0001630586,0.0001630861,0.0001629144,0.0001627017,0.0001625923,0.0001624371,0.0001623120,0.0001622334,0.0001620575,0.0001620061,0.0001619034,0.0001617865,0.0001616413,0.0001615431,0.0001614558,0.0001613383,0.0001612313,0.0001612198,0.0001612623,]
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(np.arange(len(mse_val_31)), mse_val_31, label="31 Joints (pred_steps=10)")
ax.plot(np.arange(len(mse_val_53_pred0)), mse_val_53_pred0, label="53 Joints (pred_steps=0)")
ax.plot(np.arange(len(mse_val_53)), mse_val_53, label="53 Joints (pred_steps=10)")
ax.plot(np.arange(len(mse_val_53_pred45)), mse_val_53_pred45, label="53 Joints (pred_steps=45)")
ax.set_xlabel("Epochs", fontsize=16)
ax.set_ylabel("Mean-Squared Error", fontsize=16)
# ax.set_yscale('log')
ax.legend(fontsize=16)
ax.set_title("Validation MSE during Training", fontsize=24)
ax.tick_params(labelsize=14)
ax.grid(True, linestyle='-.')
plt.savefig("mse_val.png")

In [ ]:
mse_test_31_pred10 = [ 0.000170553569 , 0.000036479261 , 0.000040822440 , 0.000043463529 , 0.000044169617 , 0.000040992756 , 0.000052417177 , 0.000049289967 , 0.000038033617 , 0.000038555027 , 0.000045348890 , 0.000053333089 , 0.000056963599 , 0.000062139421 , 0.000064282540 , 0.000072893068 , 0.000072591880 , 0.000074101619 , 0.000068075438 , 0.000058129153 , 0.000050577906 , 0.000053190113 , 0.000061028579 , 0.000059414029 , 0.000056036995 , 0.000063250787 , 0.000077031138 , 0.000081481870 , 0.000078439807 , 0.000077250144 , 0.000068747329 , 0.000073268013 , 0.000071990245 , 0.000064832428 , 0.000059746075 , 0.000065874025 , 0.000074851552 , 0.000091193571 , 0.000079429017 , 0.000062412873 , 0.000057838697 , 0.000058919220 , 0.000068225956 , 0.000068378104 , 0.008338616230 , 0.002715452807 , 0.000138973686 , 0.000076443481 , 0.000069587324 , 0.000070251663 , 0.000267180905 , 0.000599655730 , 0.001080392161 , 0.001697040512 , 0.002384907100 , 0.003133618273 , 0.003959581722 , 0.004725907929 , 0.005240121856 , 0.005618790165 , 0.005903609097 , 0.006051024888 , 0.006046074908 , 0.005888763350 , 0.005619384814 , 0.005267083179 , 0.004888665862 , 0.004478474148 , 0.004095981363 , 0.003716580570 , 0.003377792193 , 0.003085476346 , 0.002838675864 , 0.002671708586 , 0.002581292763 , 0.002571772086 , 0.002641813597 , 0.002715498442 , 0.002788403770 , 0.002859796630 , 0.002959491452 , 0.003105047159 , 0.003314523725 , 0.003548058448 , 0.003760647960 , 0.003897912567 , 0.003918492235 , 0.003850779263 , 0.003749305382 , 0.003660943359 , 0.003603305900 , 0.003591219895 , 0.003625167534 , 0.003681835718 , 0.003741636639 , 0.003827380016 , 0.003916278481 , 0.003977367189 , 0.003991694655 , 0.003950018436 , 0.003862826386 , 0.003766747890 , 0.003685059026 , 0.003635568544 , 0.003633951070 , 0.003685781965 , 0.003783362685 , 0.003910547588 , 0.004027982708 , 0.004111268092 , 0.004165213089 , 0.004194930661 , 0.004214771558 , 0.004241296090 , 0.004284715746 , 0.004346783273 , 0.004427754320 , 0.004530124832 , 0.004630494863 , 0.004697221797 , 0.004701348022 , 0.004612467717 , 0.004461598117 , 0.004302240442 , 0.004188369494 , 0.004188398365 , 0.004342246801 , 0.004671652336 , 0.005178018939 , 0.005791888107 , 0.006389168557 , 0.006796973292 , 0.006966274232 , 0.006945447531 , 0.006649866235 , 0.006354937330 , 0.006095191929 , 0.005853790790 , 0.005586397368 , 0.005279938225 , 0.004933481570 , 0.004600205459 , 0.004318840336 , 0.004070111550 , 0.003882152261 , 0.003769358620 , 0.003712567501 , 0.003711000085 , 0.003773406846 , 0.003894917434 , 0.004088368267 , 0.004349150229 , 0.004612825345 , 0.004807871766 , 0.004894962069 , 0.004836157896 , 0.004704452120 , 0.004538103007 , 0.004340854939 , 0.004143003840 , 0.003976247273 , 0.003883879399 , 0.003918698989 , 0.004118197132 , 0.004464866128 , 0.004870582372 , 0.005255959928 , 0.005542080384 , 0.005705200601 , 0.005720876157 , 0.005705336109 , 0.005640796851 , 0.005541868508 , 0.005511490162 , 0.005529896822 , 0.005662469659 , 0.005776919425 , 0.005801928695 , 0.005734444130 , 0.005584228318 , 0.005404597614 , 0.005210887641 , 0.005072689150 , 0.005054837558 , 0.005173311569 , 0.005380696151 , 0.005649754778 , 0.005867110100 , 0.006006422918 , 0.006013969425 , 0.005942238029 , 0.005784451496 , 0.005583574530 , 0.005346609745 , 0.005112408195 , 0.004946506117 , 0.004897031933 , 0.004978809040 , 0.005128024612 , 0.005314152688 , 0.005509041250 , 0.005656914320 , 0.005770260468 , 0.005831124261 , 0.005840248894 , 0.005875584204 , 0.006023585796 , 0.006270071026 , 0.006535457447 , 0.006770545617 , 0.006900213193 , 0.006906327326 , 0.006801181938 , 0.006624378264 , 0.006428692024 , 0.006249960046 , 0.006103171501 , 0.005968021229 , 0.005828411784 , 0.005650281440 , 0.005457575899 , 0.005269198213 , 0.005162003916 , 0.005193312652 , 0.005379545037 , 0.005704376381 , 0.006078921724 , 0.006444502156 , 0.006687717512 , 0.006831370294 , 0.006785545498 , 0.006536947098 , 0.006168949883 , 0.005770295858 , 0.005447971169 , 0.005237493664 , 0.005138285458 , 0.005083232652 , 0.005074953660 , 0.005108599085 , 0.005177326035 , 0.005244163796 , 0.005290446337 , 0.005277037621 , 0.005173637997 , 0.004973222967 , 0.004719117191 , 0.004463247955 , 0.004241055343 , 0.004082195461 , 0.004004514311 , 0.004006917123 , 0.004068385344 , 0.004171139561 , 0.004285783973 , 0.004390835762 , 0.004479652736 , 0.004549563397 , 0.004613267258 , 0.004679564387 , 0.004758393858 , 0.004858015571 , 0.004984947387 , 0.005116302520 , 0.005228862632 , 0.005327943247 , 0.005416995380 , 0.005485721398 , 0.005520852748 , 0.005509871989 , 0.005466484930 , 0.005396574736 , 0.005295308307 , 0.005213181023 , 0.005170595367 , 0.005173195153 , 0.005191018805 , 0.005230549257 , 0.005312772933 , 0.005411883350 , 0.005509941373 , 0.005625264719 , 0.005712520797 , 0.005773853976 , 0.005799907725 , 0.005799675826 , 0.005795020610 , 0.005794547964 , 0.005816301331 , 0.005809693132 , 0.005738581065 , 0.005603366066 , 0.005474871490 , 0.005356660113 , 0.005305234808 , 0.005357537419 , 0.005541501101 , 0.005828235298 , 0.006160370540 , 0.006489444058 , 0.006777709816 , 0.007017849013 , 0.007162588183 , 0.007243693341 , 0.007268721703 , 0.007273069117 , 0.007257214282 , 0.007271451876 , 0.007344617043 , 0.007432248909 , 0.007505256217 , 0.007635687012 , 0.007826889865 , 0.008053523488 , 0.008276012726 , 0.008461476304 , 0.008594642393 , 0.008638492785 , 0.008599733002 , 0.008543414064 , 0.008557151072 , 0.008669530042 , 0.008997793309 , 0.009453360923 , 0.009930006228 , 0.010382063687 , 0.010758574121 , 0.010964263231 , 0.011022734456 , 0.010949749500 , 0.010697737336 , 0.010358459316 , 0.010008882731 , 0.009694841690 , 0.009454614483 , 0.009259662591 , 0.009077809751 , 0.008930877782 , 0.008767629974 , 0.008620955981 , 0.008506179787 , 0.008411249146 , 0.008339861408 , 0.008272348903 , 0.008189314045 , 0.008100817911 , 0.007970430888 , 0.007791236043 , 0.007609905675 , 0.007479129825 , 0.007428985089 , 0.007481930312 , 0.007613698486 , 0.007794321049 , 0.007984348573 , 0.008096925914 , 0.008012522943 , 0.007860003971 , 0.007672821637 , 0.007513952907 , 0.007369938772 , 0.007270984352 , 0.007283144165 , 0.007406821474 , 0.007577490062 , 0.007705095690 , 0.007807655260 , 0.007890157402 , 0.007961832918 , 0.008035778068 , 0.008075723425 , 0.008020187728 , 0.007876222022 , 0.007749833167 , 0.007684538607 , 0.007674401160 , 0.007722310256 , 0.007776960731 , 0.007783622015 , 0.007739037741 , 0.007654642221 , 0.007535761222 , 0.007411511615 , 0.007281679194 , 0.007178331260 , 0.007134184241 , 0.007152207196 , 0.007200968917 , 0.007279476617 , 0.007347682957 , 0.007396850735 , 0.007437021937 , 0.007470312063 , 0.007523513865 , 0.007594242692 , 0.007677088957 , 0.007704025600 , 0.007587235421 , 0.007455751766 , 0.007335132454 , 0.007218281273 , 0.007101702038 , 0.007026423234 , 0.007008173037 , 0.007055189926 , 0.007177484687 , 0.007317150012 , 0.007437134627 , 0.007526934147 , 0.007617170457 , 0.007702567149 , 0.007728290278 , 0.007737652864 , 0.007774934638 , 0.007807014976 , 0.007839507423 , 0.007917716168 , 0.007975749671 , 0.007973191328 , 0.007962965406 , 0.007924071513 , 0.007859882899 , 0.007781987544 , 0.007713209838 , 0.007747276220 , 0.007819305174 , 0.007799320389 , 0.007769580465 , 0.007777051535 , 0.007862258703 , 0.008004124276 , 0.008167098276 , 0.008276350796 , 0.008304022253 , 0.008252922446 , 0.008151694201 , 0.008045807481 , 0.007945162244 , 0.007870115340 , 0.007844689302 , 0.007859049365 , 0.007867828943 , 0.007829698734 , 0.007758753840 , 0.007676098961 , 0.007591455709 , 0.007531674579 , 0.007469006348 , 0.007426579949 , 0.007411592174 , 0.007433067542 , 0.007496695500 , 0.007595663890 , 0.007750684861 , 0.007960943505 , 0.008229446597 , 0.008548160084 , 0.008878057823 , 0.009197763167 , 0.009510640055 , 0.009761848487 , 0.010003854521 , 0.010204833001 , 0.010359597392 , 0.010487119667 , 0.010611121543 , 0.010599008761 , 0.010467627086 , 0.010299097747 , 0.010157189332 , 0.010087724775 , 0.010048454627 , 0.010013931431 , 0.009894457646 , 0.009741659276 , 0.009650350548 , 0.009631290101 , 0.009669902734 , 0.009768266231 , 0.009895493276 , 0.009987587109 , 0.009996839799 , 0.009902944788 , 0.009718026035 , 0.009472267702 , 0.009195149876 , 0.008920274675 , 0.008712441660 , 0.008560626768 , 0.008486174978 , 0.008413190953 , 0.008350860327 , 0.008346180432 , 0.008400391787 , 0.008426158689 , 0.008452654816 , 0.008468545042 , 0.008502532728 , 0.008534806781 , 0.008569065481 , 0.008563269861 , 0.008519825526 , 0.008397330530 , 0.008260299452 , 0.008077877574 , 0.007884352468 , 0.007681226358 , 0.007462305948 , 0.007306495681 , 0.007200654130 , 0.007137407083 , 0.007126502693 , 0.007155373227 , 0.007224927191 , 0.007311120164 , 0.007403897122 , 0.007518933620 , 0.007700237911 , 0.007888696156 , 0.008059778251 , 0.008247091435 , 0.008453387767 , 0.008620855398 , 0.008727143519 , 0.008743407205 , 0.008670954034 , 0.008552022278 , 0.008423078805 , 0.008349870332 , 0.008332939819 , 0.008304914460 , 0.008267954923 , 0.008183875121 , 0.007991663180 , 0.007743323222 , 0.007540582214 , 0.007425667718 , 0.007410760503 , 0.007485441864 , 0.007635308430 , 0.007798181381 , 0.007921095006 , 0.007998897694 , 0.008003694937 , 0.007926776074 , 0.007728962228 , 0.007413953077 , 0.007095304783 , 0.006843869109 , 0.006751719397 , 0.006869588047 , 0.007204039488 , 0.007605716586 , 0.007990638725 , 0.008246169426 , 0.008367515169 , 0.008414970711 , 0.008363018744 , 0.008217308670 , 0.008092286997 , 0.008015980013 , 0.008017792366 , 0.008095373400 , 0.008203906007 , 0.008302775212 , 0.008325349540 , 0.008282166906 , 0.008178922348 , 0.007997783832 , 0.007795911282 , 0.007596816402 , 0.007439382840 , 0.007370837498 , 0.007403234020 , 0.007509995252 , 0.007642904297 , 0.007753312122 , 0.007842238992 , 0.007889755070 , 0.007922145538 , 0.007930258289 , 0.007909008302 , 0.007919557393 , 0.007971845567 , 0.008079953492 , 0.008264712058 , 0.008493693545 , 0.008677596226 , 0.008748668246 , 0.008750741370 , 0.008657936007 , 0.008536967449 , 0.008403012529 , 0.008297546767 , 0.008249185048 , 0.008268518373 , 0.008325468749 , 0.008412362076 , 0.008494063281 , 0.008492949419 , 0.008382045664 , 0.008218940347 , 0.008056514896 , 0.007940236479 , 0.007865861058 , 0.007828622125 , 0.007771167438 , 0.007698459551 , 0.007593320683 , 0.007463794202 , 0.007284922991 , 0.007056785282 , 0.006801842246 , 0.006559902336 , 0.006345419213 , 0.006162955891 , 0.006008854136 , 0.005903505255 , 0.005840961356 , 0.005810563918 , 0.005810026545 , 0.005813723430 , 0.005821791012 , 0.005842255428 , 0.005877333228 , 0.005929468200 , 0.005988245364 , 0.006032176316 , 0.006076978520 , 0.006134770811 , 0.006202587858 , 0.006282802671 , 0.006355986465 , 0.006403710227 , 0.006418112665 , 0.006389116403 , 0.006332907360 , 0.006253965665 , 0.006156897172 , 0.006056234706 , 0.005961871240 , 0.005891928449 , 0.005855792668 , 0.005857456010 , 0.005904301535 , 0.005990493577 , 0.006112678442 , 0.006282892078 , 0.006486963481 , 0.006697251927 , 0.006903475150 , 0.007064521313 , 0.007174704224 , 0.007249841467 , 0.007308015600 , 0.007374510635 , 0.007465312723 , 0.007633492351 , 0.007924987003 , 0.008137957193 , 0.008262983523 , 0.008337308653 , 0.008460315876 , 0.008474518545 , 0.008213626221 , 0.007986184210 , 0.007781368215 , 0.007504059467 , 0.007296601776 , 0.007184654009 , 0.007142698858 , 0.007148450240 , 0.007199608255 , 0.007306057494 , 0.007440913934 , 0.007565621287 , 0.007669763174 , 0.007739420515 , 0.007790557574 , 0.007860449143 , 0.007990294136 , 0.008239631541 , 0.008575343527 , 0.008980768733 , 0.009411910549 , 0.009701500647 , 0.009747634642 , 0.009554866701 , 0.009256622754 , 0.008897514082 , 0.008552620187 , 0.008348279633 , 0.008156368509 , 0.007930460386 , 0.007700624410 , 0.007521734107 , 0.007367579266 , 0.007225596812 , 0.007108281832 , 0.007025534287 , 0.006997145712 , 0.006971742958 , 0.006925661582 , 0.006825218443 , 0.006798598450 , 0.006839446258 , 0.006952300202 , 0.007129901554 , 0.007298976649 , 0.007414729800 , 0.007439898793 , 0.007362555712 , 0.007195716258 , 0.006975151133 , 0.006792665925 , 0.006736749783 , 0.006804265548 , 0.006999501493 , 0.007281757426 , 0.007581187412 , 0.007800165564 , 0.007869734429 , 0.007824690081 , 0.007699336857 , 0.007533532102 , 0.007370095234 , 0.007254768163 , 0.007215248886 , 0.007260037120 , 0.007420892362 , 0.007678033784 , 0.007935048081 , 0.008144543506 , 0.008292472921 , 0.008386145346 , 0.008450080641 , 0.008472182788 , 0.008484961465 , 0.008437845856 , 0.008330903947 , 0.008204900660 , 0.008110112511 , 0.008034161292 , 0.008041607216 , 0.008136020042 , 0.008332417347 , 0.008619087748 , 0.008991801180 , 0.009405005723 , 0.009774745442 , 0.010038710199 , 0.010221156292 , 0.010261409916 , 0.010143954307 , 0.009869094007 , 0.009556521662 , 0.009240319021 , 0.008938190527 , 0.008719640784 , 0.008531169966 , 0.008323885500 , 0.008100843988 , 0.007864660583 , 0.007597097661 , 0.007340301294 , 0.007149273064 , 0.007051581983 , 0.007051091176 , 0.007188920397 , 0.007374055684 , 0.007540347520 , 0.007641690318 , 0.007714151870 , 0.007778367493 , 0.007825548761 , 0.007887620479 , 0.007928544655 , 0.007945883088 , 0.007938555442 , 0.007949021645 , 0.007954569533 , 0.007918462157 , 0.007845400833 , 0.007746641058 , 0.007622591686 , 0.007512181532 , 0.007433187217 , 0.007387347054 , 0.007396470290 , 0.007432429120 , 0.007447356358 , 0.007410428952 , 0.007353974972 , 0.007292411756 , 0.007222127169 , 0.007173693273 , 0.007173184771 , 0.007217967417 , 0.007256169338 , 0.007208785042 , 0.007069563493 , 0.006900901441 , 0.006720133591 , 0.006536924746 , 0.006355239544 , 0.006197841838 , 0.006091339514 , 0.006013408303 , 0.005974921864 , 0.005985103082 , 0.006065045949 , 0.006190802436 , 0.006323799025 , 0.006459010765 , 0.006572363898 , 0.006708361674 , 0.006877664011 , 0.007017755881 , 0.007095842157 , 0.007136818487 , 0.007142890245 , 0.007120572496 , 0.007083183154 , 0.007021110039 , 0.006934382487 , 0.006845145021 , 0.006788857281 , 0.006771317217 , 0.006726583000 , 0.006708484609 , 0.006709770765 , 0.006748711690 , 0.006818668917 , 0.006917115301 , 0.007062016521 , 0.007200152148 , 0.007287810091 , 0.007348610088 , 0.007391697261 , 0.007383523509 , 0.007384150755 , 0.007422947790 , 0.007490247022 , 0.007602609228 , 0.007740763947 , 0.007897648960 , 0.008030617610 , 0.008055279031 , 0.007960657589 , 0.007749261800 , 0.007496172562 , 0.007277537137 , 0.007142573595 , 0.007106854115 , 0.007099511568 , 0.007018608972 , 0.006900090724 , 0.006777292117 , 0.006673119962 , 0.006594820414 , 0.006640613545 , 0.006842759904 , 0.007184579503 , 0.007549019065 , 0.007803229615 , 0.007904545404 , 0.007850473747 , 0.007717758417 , 0.007590657100 , 0.007538292091 , 0.007541363593 , 0.007607510313 , 0.007689365651 , 0.007684424520 , 0.007571982220 , 0.007459720131 , 0.007391395513 , 0.007398194168 , 0.007524126675 , 0.007717344444 , 0.007873045281 , 0.007974472828 , 0.007996194996 , 0.007935622707 , 0.007783067878 , 0.007564637810 , 0.007342707831 , 0.007203242276 , 0.007188235875 , 0.007231977303 , 0.007313694339 , 0.007360990625 , 0.007390145678 , 0.007393377367 , 0.007400015369 , 0.007371445652 , 0.007369754370 , 0.007415622473 , 0.007467908785 , 0.007483997848 , 0.007467481773 , 0.007399021182 , 0.007370256353 , 0.007417718414 , 0.007587016094 , 0.007881227881 , 0.008260750212 , 0.008653886616 , 0.009011019953 , 0.009260846302 , 0.009423450567 , 0.009467351250 , 0.009380665608 , 0.009182495996 , 0.008941159584 , 0.008708956651 , 0.008489800617 , 0.008297023363 , 0.008133254014 , 0.007994656451 , 0.007868577726 , 0.007761447225 , 0.007642127573 , 0.007519213948 , 0.007405275945 , 0.007291780319 , 0.007177297026 , 0.007046649698 , 0.006895961240 , 0.006753474474 , 0.006629869342 , 0.006596548017 , 0.006694267970 , 0.006915127393 , 0.007218501996 , 0.007520943414 , 0.007660866249 , 0.007579703350 , 0.007385749370 , 0.007109112572 , 0.006797553971 , 0.006519673858 , 0.006309955847 , 0.006202557590 , 0.006222620141 , 0.006317235529 , 0.006396427751 , 0.006430726498 , 0.006422702689 , 0.006356822792 , 0.006247233134 , 0.006144572515 , 0.006050650030 , 0.005989396945 , 0.005974240601 , 0.006018170621 , 0.006089590956 , 0.006163176149 , 0.006260498893 , 0.006338343490 , 0.006460167002 , 0.006633054465 , 0.006819529459 , 0.006985731423 , 0.007108115125 , 0.007203742396 , 0.007202018052 , 0.007118727546 , 0.006973500829 , 0.006796620786 , 0.006635516416 , 0.006538584363 , 0.006504687015 , 0.006481193472 , 0.006430957001 , 0.006368000526 , 0.006290924270 , 0.006210983265 , 0.006167802494 , 0.006185011473 , 0.006274797022 , 0.006434602663 , 0.006621066015 , 0.006778454874 , 0.006842833478 , 0.006823602598 , 0.006750149652 , 0.006687439512 , 0.006695318501 , 0.006808062550 , 0.007036555558 , 0.007383607328 , 0.007754210848 , 0.008077909239 , 0.008387216367 , 0.008632626384 , 0.008775315247 , 0.008834539913 , 0.008797896095 ]
mse_test_53_pred10 = [ 0.000158570459 , 0.000051685260 , 0.000053749940 , 0.000056186440 , 0.000057380574 , 0.000053155021 , 0.000060354738 , 0.000061248902 , 0.000052378735 , 0.000051323193 , 0.000054035467 , 0.000060706563 , 0.000066055836 , 0.000074103111 , 0.000079687248 , 0.000090338486 , 0.000090451009 , 0.000093248011 , 0.000085932777 , 0.000077319957 , 0.000066654298 , 0.000066013818 , 0.000069924507 , 0.000070782793 , 0.000072261661 , 0.000080675942 , 0.000095696916 , 0.000098401797 , 0.000098365505 , 0.000099267178 , 0.000092988812 , 0.000095058371 , 0.000094943716 , 0.000088235647 , 0.000079720761 , 0.000078441022 , 0.000078358709 , 0.000084072766 , 0.000082334154 , 0.000070912160 , 0.000062767162 , 0.000058664056 , 0.000061312858 , 0.000068760521 , 0.008294674568 , 0.005455928389 , 0.000114321425 , 0.000096234005 , 0.000091984431 , 0.000086858658 , 0.000317631057 , 0.000673240691 , 0.001132978126 , 0.001674420200 , 0.002255576430 , 0.002856722102 , 0.003497446189 , 0.004080508836 , 0.004444147460 , 0.004638380371 , 0.004763920791 , 0.004839734640 , 0.004867909476 , 0.004810537677 , 0.004693099763 , 0.004498738330 , 0.004237504210 , 0.003911338281 , 0.003567910520 , 0.003216735786 , 0.002904707799 , 0.002643982880 , 0.002424559090 , 0.002255716128 , 0.002154551679 , 0.002129655331 , 0.002195715904 , 0.002291457960 , 0.002401177073 , 0.002500377363 , 0.002620104933 , 0.002776464215 , 0.002968353452 , 0.003183012130 , 0.003347364487 , 0.003437999869 , 0.003422338748 , 0.003331365297 , 0.003201965010 , 0.003059693379 , 0.002933731070 , 0.002836357569 , 0.002773932181 , 0.002749469364 , 0.002757384442 , 0.002806505188 , 0.002885456430 , 0.002946787281 , 0.002968551591 , 0.002946836175 , 0.002887652256 , 0.002811231883 , 0.002734517911 , 0.002681434387 , 0.002667346969 , 0.002708180109 , 0.002798857400 , 0.002924272092 , 0.003052108688 , 0.003146422096 , 0.003202290507 , 0.003241005354 , 0.003266449785 , 0.003306229599 , 0.003364155069 , 0.003434393322 , 0.003517404199 , 0.003599073505 , 0.003671439132 , 0.003701496869 , 0.003671812592 , 0.003558061086 , 0.003394283121 , 0.003213811899 , 0.003076946130 , 0.003051999956 , 0.003189866431 , 0.003505343804 , 0.003959483933 , 0.004493053537 , 0.005030236673 , 0.005423124880 , 0.005599427503 , 0.005502494052 , 0.005190132186 , 0.004869048949 , 0.004551256076 , 0.004249207210 , 0.003955204971 , 0.003662579460 , 0.003353765933 , 0.003113675863 , 0.002945797518 , 0.002841522219 , 0.002799113281 , 0.002807744080 , 0.002839788795 , 0.002887927229 , 0.002961447462 , 0.003061005846 , 0.003208022332 , 0.003408849472 , 0.003625103040 , 0.003779565915 , 0.003846782958 , 0.003804682521 , 0.003709080396 , 0.003589894623 , 0.003455385333 , 0.003324927529 , 0.003225689754 , 0.003183396533 , 0.003234454663 , 0.003408723278 , 0.003688888624 , 0.004011112731 , 0.004320580047 , 0.004550220910 , 0.004705320112 , 0.004763897508 , 0.004816701636 , 0.004841861781 , 0.004840750713 , 0.004867438227 , 0.004919204861 , 0.005039352458 , 0.005139185116 , 0.005143884104 , 0.005048029590 , 0.004872122314 , 0.004667836707 , 0.004460228141 , 0.004300898407 , 0.004246741533 , 0.004322099034 , 0.004474908113 , 0.004665420391 , 0.004826676566 , 0.004921742249 , 0.004927459639 , 0.004877543543 , 0.004758944269 , 0.004592991900 , 0.004389014095 , 0.004159839358 , 0.003974927589 , 0.003898669966 , 0.003938296810 , 0.004053613171 , 0.004203372169 , 0.004346541595 , 0.004455195274 , 0.004533396568 , 0.004577318672 , 0.004611289594 , 0.004711510148 , 0.004935473204 , 0.005246831570 , 0.005589234177 , 0.005914606620 , 0.006124916952 , 0.006199100520 , 0.006142970640 , 0.006011689082 , 0.005861185957 , 0.005723942071 , 0.005601408426 , 0.005471870303 , 0.005314836744 , 0.005102745723 , 0.004839770030 , 0.004546822514 , 0.004315902013 , 0.004240735434 , 0.004330384079 , 0.004567511845 , 0.004855095409 , 0.005132579710 , 0.005291505251 , 0.005337545183 , 0.005181398243 , 0.004855254199 , 0.004449156579 , 0.004039709922 , 0.003720545908 , 0.003515851684 , 0.003425101750 , 0.003389760619 , 0.003388303332 , 0.003420674242 , 0.003479235107 , 0.003527851775 , 0.003555970965 , 0.003530941904 , 0.003438492306 , 0.003254741430 , 0.003015863011 , 0.002766048769 , 0.002547077369 , 0.002388284774 , 0.002300536959 , 0.002298603766 , 0.002364795422 , 0.002477609785 , 0.002625488909 , 0.002767241560 , 0.002892544260 , 0.002983676502 , 0.003047593171 , 0.003093756735 , 0.003140898421 , 0.003203034168 , 0.003288876265 , 0.003400620772 , 0.003512290306 , 0.003623231081 , 0.003724000417 , 0.003801495768 , 0.003845466301 , 0.003840597114 , 0.003808457172 , 0.003742980771 , 0.003650399623 , 0.003566317959 , 0.003489430062 , 0.003441387089 , 0.003403052455 , 0.003387752920 , 0.003394326195 , 0.003415021114 , 0.003446196206 , 0.003486650065 , 0.003514696145 , 0.003515195800 , 0.003478488885 , 0.003435147926 , 0.003403814742 , 0.003399968846 , 0.003443682566 , 0.003500499995 , 0.003491010284 , 0.003393946914 , 0.003271426773 , 0.003126895986 , 0.003025355050 , 0.003012968227 , 0.003111424157 , 0.003281691344 , 0.003486092435 , 0.003682366339 , 0.003845795291 , 0.003978515975 , 0.004064789042 , 0.004119934980 , 0.004152423237 , 0.004179216456 , 0.004223679658 , 0.004316723440 , 0.004451988731 , 0.004603475332 , 0.004730537068 , 0.004891396966 , 0.005051676184 , 0.005209958181 , 0.005348106846 , 0.005468150135 , 0.005553429481 , 0.005604478065 , 0.005620015785 , 0.005655860528 , 0.005696948618 , 0.005851462949 , 0.006222455297 , 0.006712503731 , 0.007182954345 , 0.007569754496 , 0.007823421620 , 0.007907946594 , 0.007880733348 , 0.007760187145 , 0.007512980606 , 0.007198567037 , 0.006878784392 , 0.006597256288 , 0.006378980353 , 0.006191649940 , 0.006011812016 , 0.005863576662 , 0.005710050929 , 0.005563355982 , 0.005419720430 , 0.005264023319 , 0.005097162910 , 0.004923229106 , 0.004755981266 , 0.004604195710 , 0.004431136418 , 0.004236370325 , 0.004062297754 , 0.003945079166 , 0.003913218621 , 0.003970827442 , 0.004085895140 , 0.004233777523 , 0.004383765161 , 0.004461673554 , 0.004361937754 , 0.004178298172 , 0.003944528289 , 0.003733726218 , 0.003553223563 , 0.003421111731 , 0.003409812227 , 0.003526601708 , 0.003709007055 , 0.003884685226 , 0.004035572987 , 0.004167708103 , 0.004310522694 , 0.004460445605 , 0.004542888608 , 0.004469031468 , 0.004198916722 , 0.003895701841 , 0.003644163022 , 0.003479463980 , 0.003417130560 , 0.003421302186 , 0.003458816558 , 0.003477401799 , 0.003490559524 , 0.003479803214 , 0.003455443541 , 0.003398356261 , 0.003328696126 , 0.003270344576 , 0.003239260288 , 0.003244974883 , 0.003289707005 , 0.003353361972 , 0.003419075161 , 0.003489301307 , 0.003564360784 , 0.003642308759 , 0.003724656999 , 0.003801060608 , 0.003803367494 , 0.003663705662 , 0.003502138657 , 0.003355953610 , 0.003224316286 , 0.003084107535 , 0.002972092479 , 0.002917716280 , 0.002913152566 , 0.002969444729 , 0.003051857464 , 0.003137703985 , 0.003183504799 , 0.003237025812 , 0.003284331178 , 0.003310062690 , 0.003334059380 , 0.003401027061 , 0.003486720147 , 0.003575926879 , 0.003664626041 , 0.003720745677 , 0.003708150936 , 0.003654975444 , 0.003573595313 , 0.003483115928 , 0.003394981148 , 0.003357651876 , 0.003418619512 , 0.003470258787 , 0.003465140238 , 0.003450708697 , 0.003443940775 , 0.003459511558 , 0.003509522183 , 0.003575935028 , 0.003612224245 , 0.003602189710 , 0.003548074514 , 0.003492359770 , 0.003460967215 , 0.003452460049 , 0.003462941153 , 0.003515319899 , 0.003586227307 , 0.003645082004 , 0.003646384692 , 0.003622974269 , 0.003590313951 , 0.003561548889 , 0.003548949258 , 0.003541883081 , 0.003542416729 , 0.003560733283 , 0.003601124743 , 0.003664831398 , 0.003743214766 , 0.003827403300 , 0.003924227320 , 0.004036022816 , 0.004168751184 , 0.004303792957 , 0.004470295738 , 0.004652402364 , 0.004790904466 , 0.004928317387 , 0.005030852277 , 0.005123019684 , 0.005212711170 , 0.005349246785 , 0.005427489057 , 0.005423966795 , 0.005378544796 , 0.005323001649 , 0.005294252653 , 0.005253191572 , 0.005182079505 , 0.005033400841 , 0.004897304345 , 0.004837302025 , 0.004866110627 , 0.004983133171 , 0.005180324893 , 0.005424960051 , 0.005607558880 , 0.005693329964 , 0.005674282555 , 0.005562988576 , 0.005386275705 , 0.005156668834 , 0.004925619345 , 0.004741279874 , 0.004594703205 , 0.004491319880 , 0.004407542292 , 0.004340713378 , 0.004324911628 , 0.004354516510 , 0.004387809429 , 0.004439935554 , 0.004505134653 , 0.004593810067 , 0.004690143745 , 0.004770638421 , 0.004803086165 , 0.004777263850 , 0.004671602510 , 0.004549883772 , 0.004410443362 , 0.004283543210 , 0.004137840588 , 0.003971046302 , 0.003824482439 , 0.003718955675 , 0.003664633259 , 0.003658923553 , 0.003675480140 , 0.003723100759 , 0.003792714095 , 0.003896883689 , 0.004018140957 , 0.004203850869 , 0.004391736817 , 0.004569779150 , 0.004770310130 , 0.004982894752 , 0.005139017012 , 0.005227054004 , 0.005222052801 , 0.005133017898 , 0.005008380394 , 0.004870356992 , 0.004786186386 , 0.004759171512 , 0.004714932758 , 0.004660962615 , 0.004582940601 , 0.004420576151 , 0.004201035947 , 0.003996993881 , 0.003867637366 , 0.003841833444 , 0.003895548405 , 0.004019717686 , 0.004177063238 , 0.004310339224 , 0.004401723854 , 0.004406140652 , 0.004333434161 , 0.004149409477 , 0.003864069469 , 0.003589430125 , 0.003388289362 , 0.003321133321 , 0.003419027664 , 0.003705298295 , 0.004099419340 , 0.004496484995 , 0.004750808235 , 0.004885236267 , 0.004940139595 , 0.004890725482 , 0.004761325195 , 0.004649824928 , 0.004587942269 , 0.004571991041 , 0.004627333954 , 0.004697286058 , 0.004768650513 , 0.004788481165 , 0.004765650257 , 0.004692074843 , 0.004544292577 , 0.004389379639 , 0.004255726002 , 0.004174307454 , 0.004169924185 , 0.004254484084 , 0.004399799276 , 0.004540693015 , 0.004616562743 , 0.004642714746 , 0.004643395077 , 0.004649131093 , 0.004651473369 , 0.004646994639 , 0.004626865033 , 0.004625778645 , 0.004634209443 , 0.004692991264 , 0.004816990811 , 0.004941724706 , 0.004992966540 , 0.004971010610 , 0.004860922229 , 0.004725877661 , 0.004553092178 , 0.004395171534 , 0.004274525214 , 0.004221505951 , 0.004226001445 , 0.004281162284 , 0.004376365338 , 0.004461129196 , 0.004476268776 , 0.004442567471 , 0.004367330112 , 0.004291396122 , 0.004244660959 , 0.004215042107 , 0.004147367086 , 0.004034824204 , 0.003895201720 , 0.003742226632 , 0.003568452084 , 0.003376120701 , 0.003180715954 , 0.002983191982 , 0.002800896065 , 0.002645843429 , 0.002520360285 , 0.002441810677 , 0.002408560598 , 0.002404932864 , 0.002422229620 , 0.002423697384 , 0.002420160919 , 0.002415124560 , 0.002417179057 , 0.002422125777 , 0.002430860186 , 0.002426184248 , 0.002423582366 , 0.002429621527 , 0.002452701563 , 0.002501137787 , 0.002556841820 , 0.002591780154 , 0.002599290572 , 0.002568334108 , 0.002517771209 , 0.002449511550 , 0.002367440378 , 0.002274716739 , 0.002175596077 , 0.002087833825 , 0.002021172782 , 0.001992837759 , 0.002023525070 , 0.002096909797 , 0.002212827792 , 0.002369870432 , 0.002550084377 , 0.002740777330 , 0.002930090996 , 0.003094446613 , 0.003234705655 , 0.003379485803 , 0.003524974221 , 0.003680840135 , 0.003836616175 , 0.004040375352 , 0.004308496136 , 0.004519444890 , 0.004701804370 , 0.004858938511 , 0.004998294171 , 0.004979903344 , 0.004712587688 , 0.004417159129 , 0.004099943209 , 0.003752167104 , 0.003491098294 , 0.003334121080 , 0.003260475816 , 0.003251677379 , 0.003300230950 , 0.003394237021 , 0.003514959710 , 0.003619633382 , 0.003702031448 , 0.003749869531 , 0.003808771493 , 0.003915038891 , 0.004114713054 , 0.004417416640 , 0.004781097174 , 0.005160125904 , 0.005466686562 , 0.005612904206 , 0.005570152309 , 0.005364338402 , 0.005074963439 , 0.004744561389 , 0.004427292384 , 0.004219931550 , 0.004036314785 , 0.003836131422 , 0.003668902675 , 0.003541279817 , 0.003464593319 , 0.003433695994 , 0.003447161987 , 0.003509634407 , 0.003595408052 , 0.003659850219 , 0.003664888442 , 0.003559432225 , 0.003501007566 , 0.003498415463 , 0.003584651509 , 0.003760247491 , 0.003960884642 , 0.004114267416 , 0.004160904326 , 0.004080350045 , 0.003930681851 , 0.003768699011 , 0.003676444292 , 0.003699365072 , 0.003842005506 , 0.004072262906 , 0.004357653204 , 0.004641371313 , 0.004842533264 , 0.004897819366 , 0.004823369905 , 0.004661689512 , 0.004441498779 , 0.004199336749 , 0.003971466329 , 0.003787308699 , 0.003676626598 , 0.003661804134 , 0.003720111912 , 0.003822408849 , 0.003904773854 , 0.003970100544 , 0.004012109246 , 0.004046581220 , 0.004043136723 , 0.004005203955 , 0.003897252260 , 0.003732952289 , 0.003601226257 , 0.003568002954 , 0.003618031042 , 0.003783783643 , 0.004057569895 , 0.004397667479 , 0.004770020489 , 0.005216476973 , 0.005691252183 , 0.006127002183 , 0.006465028506 , 0.006681522354 , 0.006732879672 , 0.006640676875 , 0.006378422026 , 0.006038451102 , 0.005691577215 , 0.005391760264 , 0.005217314232 , 0.005119624548 , 0.004982070066 , 0.004761994351 , 0.004457870964 , 0.004141636658 , 0.003875324503 , 0.003687834367 , 0.003625423415 , 0.003702879651 , 0.003887624480 , 0.004092579242 , 0.004271409940 , 0.004369224887 , 0.004388671368 , 0.004366443027 , 0.004324399401 , 0.004285103176 , 0.004219736438 , 0.004142665770 , 0.004081328399 , 0.004043437541 , 0.004013813566 , 0.003960635979 , 0.003880945034 , 0.003794546938 , 0.003714957973 , 0.003648829879 , 0.003606210463 , 0.003596586175 , 0.003620695556 , 0.003653818974 , 0.003647517180 , 0.003590456443 , 0.003498935141 , 0.003405058756 , 0.003317009658 , 0.003273570910 , 0.003302966245 , 0.003379059024 , 0.003465047106 , 0.003488060087 , 0.003430182114 , 0.003326462582 , 0.003196918638 , 0.003063653363 , 0.002925225766 , 0.002792351879 , 0.002687996253 , 0.002618164057 , 0.002585254377 , 0.002594575053 , 0.002647193149 , 0.002726667561 , 0.002789587015 , 0.002845445881 , 0.002898766892 , 0.002996766241 , 0.003137378953 , 0.003259930061 , 0.003320527496 , 0.003346342593 , 0.003348696744 , 0.003337409114 , 0.003327897750 , 0.003308417276 , 0.003277946962 , 0.003258426907 , 0.003269165521 , 0.003302749246 , 0.003298438853 , 0.003276243573 , 0.003248025663 , 0.003238687292 , 0.003262039274 , 0.003306092927 , 0.003376853419 , 0.003442377085 , 0.003470584052 , 0.003488482907 , 0.003498772858 , 0.003465103218 , 0.003434589365 , 0.003435569117 , 0.003468424780 , 0.003554564668 , 0.003650910920 , 0.003752853721 , 0.003834648058 , 0.003849465400 , 0.003799494123 , 0.003692994593 , 0.003567503765 , 0.003475993173 , 0.003443227848 , 0.003466132097 , 0.003487373004 , 0.003440894885 , 0.003341479460 , 0.003215604695 , 0.003083006246 , 0.002941113897 , 0.002889384748 , 0.002977890195 , 0.003176752245 , 0.003394140862 , 0.003517054953 , 0.003525958164 , 0.003463684581 , 0.003381905844 , 0.003337085014 , 0.003386820434 , 0.003500391962 , 0.003637445159 , 0.003763249842 , 0.003793179756 , 0.003691125428 , 0.003549774410 , 0.003421773436 , 0.003343284130 , 0.003349208040 , 0.003400765127 , 0.003423842369 , 0.003411917016 , 0.003375779837 , 0.003320891410 , 0.003218032653 , 0.003071166342 , 0.002913629403 , 0.002826580079 , 0.002837391803 , 0.002882462228 , 0.002949704183 , 0.002981824568 , 0.003001869423 , 0.003025046550 , 0.003096072702 , 0.003172813915 , 0.003268365050 , 0.003386675147 , 0.003485363210 , 0.003530133283 , 0.003498065053 , 0.003405074356 , 0.003336268710 , 0.003357877955 , 0.003532506526 , 0.003836452961 , 0.004191013053 , 0.004507085774 , 0.004773488734 , 0.004947427195 , 0.005057514179 , 0.005088849925 , 0.005052685272 , 0.004976259079 , 0.004900489002 , 0.004825555254 , 0.004741175566 , 0.004650188144 , 0.004536804743 , 0.004410533700 , 0.004283419345 , 0.004164641257 , 0.004036519676 , 0.003908931278 , 0.003784519387 , 0.003650137223 , 0.003507277928 , 0.003363770666 , 0.003194988007 , 0.003053610213 , 0.002952960553 , 0.002919532359 , 0.002990467474 , 0.003153078491 , 0.003367872210 , 0.003589314641 , 0.003698229557 , 0.003640809795 , 0.003496018937 , 0.003272083355 , 0.003018696560 , 0.002800327027 , 0.002639676677 , 0.002561846282 , 0.002570446115 , 0.002627873560 , 0.002652902156 , 0.002638580045 , 0.002584954025 , 0.002492491854 , 0.002375760814 , 0.002271846170 , 0.002195021370 , 0.002155246679 , 0.002160563134 , 0.002220335882 , 0.002303168876 , 0.002387637040 , 0.002463127254 , 0.002510603284 , 0.002570282668 , 0.002675162628 , 0.002806582721 , 0.002933062613 , 0.003014941700 , 0.003084623488 , 0.003082483308 , 0.003006058745 , 0.002889289288 , 0.002756417030 , 0.002647449262 , 0.002588509349 , 0.002570549259 , 0.002558562672 , 0.002518709516 , 0.002483285032 , 0.002459948650 , 0.002471550135 , 0.002523688367 , 0.002623302629 , 0.002767770318 , 0.002940295031 , 0.003095301799 , 0.003194961464 , 0.003198897699 , 0.003135117702 , 0.003039047122 , 0.002945296699 , 0.002893042052 , 0.002907046350 , 0.003015611088 , 0.003216516459 , 0.003401995637 , 0.003582954640 , 0.003747144947 , 0.003876472125 , 0.003953511361 , 0.003994688857 , 0.004005615599 ]
mse_test_53_pred0 = [ 0.000139389202 , 0.000017572491 , 0.000028076953 , 0.000022650720 , 0.000021215839 , 0.000014412991 , 0.000019629342 , 0.000015298516 , 0.000012610995 , 0.000012006606 , 0.000013630649 , 0.000014632507 , 0.000013547708 , 0.000016042164 , 0.000015418913 , 0.000021558955 , 0.000020079968 , 0.000027562806 , 0.000027417247 , 0.000025186217 , 0.000013841606 , 0.000018891817 , 0.000026146490 , 0.000021233565 , 0.000017032062 , 0.000021193357 , 0.000027152806 , 0.000022466185 , 0.000017801245 , 0.000018056191 , 0.000015143994 , 0.000019480454 , 0.000017125423 , 0.000012613317 , 0.000011601591 , 0.000017338634 , 0.000019436291 , 0.000028194909 , 0.000039426450 , 0.000042241179 , 0.000035987923 , 0.000037901198 , 0.000043593991 , 0.000039784172 , 0.008773104288 , 0.000527382072 , 0.000037007561 , 0.000026277257 , 0.000025126910 , 0.000028935110 , 0.000124735336 , 0.000305910100 , 0.000579968153 , 0.000956802920 , 0.001439313754 , 0.002057419391 , 0.002777771791 , 0.003462915076 , 0.004028931726 , 0.004511054605 , 0.004928787705 , 0.005272839684 , 0.005534197669 , 0.005735485349 , 0.005918057170 , 0.006099325139 , 0.006255892571 , 0.006340170745 , 0.006365488749 , 0.006301886868 , 0.006186528131 , 0.006042596418 , 0.005878309254 , 0.005721117835 , 0.005584323313 , 0.005476214457 , 0.005393204745 , 0.005300767254 , 0.005220694467 , 0.005170673598 , 0.005173432175 , 0.005237782374 , 0.005373848137 , 0.005543810781 , 0.005719962064 , 0.005881125573 , 0.006021764129 , 0.006157400087 , 0.006274171639 , 0.006371594965 , 0.006466105580 , 0.006553867366 , 0.006633656565 , 0.006732440088 , 0.006841871887 , 0.006992093753 , 0.007160181180 , 0.007330058608 , 0.007470819633 , 0.007566700224 , 0.007631205022 , 0.007682165131 , 0.007719683927 , 0.007759252097 , 0.007805633824 , 0.007893129252 , 0.008032565005 , 0.008211325854 , 0.008408865891 , 0.008604359813 , 0.008805585094 , 0.009021987207 , 0.009265979752 , 0.009540493600 , 0.009865405969 , 0.010195538402 , 0.010536488146 , 0.010871756822 , 0.011192698963 , 0.011468008161 , 0.011680193245 , 0.011797224171 , 0.011832769029 , 0.011813866906 , 0.011727861129 , 0.011597871780 , 0.011450004764 , 0.011327550747 , 0.011268517934 , 0.011309850030 , 0.011462565511 , 0.011663624085 , 0.011823661625 , 0.011880313046 , 0.011807900853 , 0.011708624661 , 0.011595808901 , 0.011449349113 , 0.011289141141 , 0.011110521853 , 0.010938674212 , 0.010850568302 , 0.010851532221 , 0.010904312134 , 0.010991905816 , 0.011112235487 , 0.011251416057 , 0.011401557364 , 0.011578910984 , 0.011772410013 , 0.011997277848 , 0.012282643467 , 0.012601607479 , 0.012913894840 , 0.013159788214 , 0.013328049332 , 0.013473271392 , 0.013612062670 , 0.013739871792 , 0.013874806464 , 0.014030032791 , 0.014202844352 , 0.014396903105 , 0.014601686038 , 0.014804386534 , 0.014986653812 , 0.015130721964 , 0.015201922506 , 0.015217687003 , 0.015143926255 , 0.015040160157 , 0.014904327691 , 0.014718544669 , 0.014537595212 , 0.014382418245 , 0.014315702952 , 0.014252573252 , 0.014127384871 , 0.013943078928 , 0.013740289956 , 0.013526357710 , 0.013301559724 , 0.013110212050 , 0.013004123233 , 0.013014486991 , 0.013116765767 , 0.013338060118 , 0.013616763987 , 0.013921450824 , 0.014199382626 , 0.014452918433 , 0.014649509452 , 0.014816802926 , 0.014933166094 , 0.014947701246 , 0.014897323214 , 0.014821483754 , 0.014744315296 , 0.014662212692 , 0.014611139894 , 0.014639343135 , 0.014749462716 , 0.014888361096 , 0.015019118786 , 0.015143091790 , 0.015302669257 , 0.015561270528 , 0.015932152048 , 0.016346113756 , 0.016818512231 , 0.017258051783 , 0.017612898722 , 0.017860407010 , 0.017979906872 , 0.018011977896 , 0.018012400717 , 0.018018746749 , 0.017975689843 , 0.017846608534 , 0.017596052960 , 0.017266524956 , 0.016854153946 , 0.016449615359 , 0.016110861674 , 0.015893260017 , 0.015816589817 , 0.015834292397 , 0.015931487083 , 0.016009090468 , 0.016107838601 , 0.016106821597 , 0.016049684957 , 0.015957159922 , 0.015849607065 , 0.015789348632 , 0.015798268840 , 0.015848563984 , 0.015922522172 , 0.016010547057 , 0.016113797203 , 0.016240715981 , 0.016362378374 , 0.016454687342 , 0.016512038186 , 0.016495347023 , 0.016390033066 , 0.016205852851 , 0.015999918804 , 0.015795229003 , 0.015622090548 , 0.015487802215 , 0.015421636403 , 0.015425387770 , 0.015496071428 , 0.015629909933 , 0.015792272985 , 0.015968350694 , 0.016142698005 , 0.016301328316 , 0.016417695209 , 0.016508726403 , 0.016582861543 , 0.016659496352 , 0.016715332866 , 0.016756983474 , 0.016818301752 , 0.016925172880 , 0.017103224993 , 0.017300738022 , 0.017485041171 , 0.017621403560 , 0.017658991739 , 0.017598459497 , 0.017481589690 , 0.017317557707 , 0.017164099962 , 0.017002234235 , 0.016868377104 , 0.016778936610 , 0.016702702269 , 0.016626976430 , 0.016550730914 , 0.016467709094 , 0.016360791400 , 0.016260361299 , 0.016178047284 , 0.016114778817 , 0.016052877530 , 0.016033932567 , 0.016080720350 , 0.016143696383 , 0.016155151650 , 0.016166105866 , 0.016201803461 , 0.016252687201 , 0.016338756308 , 0.016495715827 , 0.016692643985 , 0.016905331984 , 0.017144322395 , 0.017369050533 , 0.017565650865 , 0.017693756148 , 0.017803920433 , 0.017906719819 , 0.017981518060 , 0.018043272197 , 0.018146963790 , 0.018261386082 , 0.018385594711 , 0.018497517332 , 0.018627116457 , 0.018751328811 , 0.018850147724 , 0.018864257261 , 0.018805481493 , 0.018652120605 , 0.018427046016 , 0.018161389977 , 0.017899950966 , 0.017713654786 , 0.017711959779 , 0.017944704741 , 0.018333626911 , 0.018797188997 , 0.019307227805 , 0.019779196009 , 0.020186031237 , 0.020559918135 , 0.020855801180 , 0.020988898352 , 0.020991561934 , 0.020896568894 , 0.020764967427 , 0.020622087643 , 0.020478280261 , 0.020343204960 , 0.020250188187 , 0.020163636655 , 0.020086439326 , 0.019966525957 , 0.019794097170 , 0.019568944350 , 0.019320650026 , 0.019085224718 , 0.018906230107 , 0.018751425669 , 0.018605500460 , 0.018441308290 , 0.018235232681 , 0.018070481718 , 0.017971189693 , 0.017905121669 , 0.017891345546 , 0.017945997417 , 0.017999682575 , 0.017975686118 , 0.017864698544 , 0.017651626840 , 0.017385113984 , 0.017077846453 , 0.016775628552 , 0.016547437757 , 0.016411146149 , 0.016373531893 , 0.016429414973 , 0.016558904201 , 0.016772834584 , 0.017100309953 , 0.017526393756 , 0.017937684432 , 0.018199702725 , 0.018161533400 , 0.018008846790 , 0.017767438665 , 0.017476685345 , 0.017190763727 , 0.016929505393 , 0.016705410555 , 0.016495117918 , 0.016311088577 , 0.016148870811 , 0.016065306962 , 0.016069700941 , 0.016170447692 , 0.016282292083 , 0.016359463334 , 0.016403740272 , 0.016445936635 , 0.016465499997 , 0.016455003992 , 0.016412865371 , 0.016362449154 , 0.016326235607 , 0.016309166327 , 0.016312904656 , 0.016290990636 , 0.016213953495 , 0.016178959981 , 0.016169298440 , 0.016137192026 , 0.016064455733 , 0.015973577276 , 0.015871716663 , 0.015790533274 , 0.015735927969 , 0.015697013587 , 0.015666313469 , 0.015655821189 , 0.015711279586 , 0.015768893063 , 0.015834888443 , 0.015913421288 , 0.016033718362 , 0.016178881750 , 0.016301764175 , 0.016414694488 , 0.016511458904 , 0.016514327377 , 0.016468586400 , 0.016368532553 , 0.016231043264 , 0.016095411032 , 0.015984673053 , 0.015937993303 , 0.015858581290 , 0.015755483881 , 0.015684323385 , 0.015651391819 , 0.015678385273 , 0.015762910247 , 0.015889104456 , 0.016015831381 , 0.016115890816 , 0.016212629154 , 0.016336360946 , 0.016486601904 , 0.016641251743 , 0.016825355589 , 0.017074484378 , 0.017368881032 , 0.017657987773 , 0.017875133082 , 0.018049657345 , 0.018174225464 , 0.018255036324 , 0.018322112039 , 0.018354723230 , 0.018306033686 , 0.018215479329 , 0.018074564636 , 0.017907675356 , 0.017723781988 , 0.017563156784 , 0.017418023199 , 0.017279250547 , 0.017158081755 , 0.017061298713 , 0.016982845962 , 0.016921442002 , 0.016866674647 , 0.016871845350 , 0.016902117059 , 0.016987545416 , 0.017107052729 , 0.017250446603 , 0.017334731296 , 0.017372503877 , 0.017400933430 , 0.017465069890 , 0.017574531958 , 0.017685467377 , 0.017732312903 , 0.017676657066 , 0.017613194883 , 0.017628714442 , 0.017702210695 , 0.017845991999 , 0.018031707034 , 0.018278365955 , 0.018516905606 , 0.018744239584 , 0.018861569464 , 0.018888594583 , 0.018793849275 , 0.018620515242 , 0.018400479108 , 0.018212620169 , 0.018068050966 , 0.017979769036 , 0.017879463732 , 0.017770256847 , 0.017685081810 , 0.017620174214 , 0.017525708303 , 0.017454290763 , 0.017399119213 , 0.017346752807 , 0.017274867743 , 0.017217172310 , 0.017171012238 , 0.017107095569 , 0.017041614279 , 0.016956953332 , 0.016868153587 , 0.016793062910 , 0.016707601026 , 0.016633702442 , 0.016587039456 , 0.016555106267 , 0.016504649073 , 0.016439689323 , 0.016350649297 , 0.016277702525 , 0.016247706488 , 0.016282761469 , 0.016369434074 , 0.016554437578 , 0.016765976325 , 0.017011672258 , 0.017318146303 , 0.017673807219 , 0.018001489341 , 0.018281670287 , 0.018490297720 , 0.018589815125 , 0.018592942506 , 0.018523925915 , 0.018438898027 , 0.018408900127 , 0.018432995304 , 0.018527431414 , 0.018642252311 , 0.018699800596 , 0.018625160679 , 0.018475649878 , 0.018351761624 , 0.018307995051 , 0.018295850605 , 0.018352454528 , 0.018423886970 , 0.018511949107 , 0.018634779379 , 0.018787456676 , 0.018925722688 , 0.018992515281 , 0.018919648603 , 0.018763164058 , 0.018571157008 , 0.018420867622 , 0.018326880410 , 0.018339270726 , 0.018436957151 , 0.018595548347 , 0.018729142845 , 0.018821327016 , 0.018894445151 , 0.018925713375 , 0.018917629495 , 0.018916180357 , 0.018948467448 , 0.019012462348 , 0.019151004031 , 0.019329989329 , 0.019563028589 , 0.019783545285 , 0.019955338910 , 0.020051302388 , 0.020054945722 , 0.020048085600 , 0.020064635202 , 0.020117966458 , 0.020190645009 , 0.020290302113 , 0.020416047424 , 0.020548293367 , 0.020631685853 , 0.020693695173 , 0.020738912746 , 0.020775420591 , 0.020798278973 , 0.020804913715 , 0.020800292492 , 0.020795151591 , 0.020736278966 , 0.020644871518 , 0.020522270352 , 0.020399663597 , 0.020258864388 , 0.020121475682 , 0.019961709157 , 0.019813047722 , 0.019655739889 , 0.019533224404 , 0.019431658089 , 0.019330887124 , 0.019252751023 , 0.019208865240 , 0.019165931270 , 0.019112424925 , 0.019021833315 , 0.018914448097 , 0.018776645884 , 0.018664831296 , 0.018622228876 , 0.018614551052 , 0.018608154729 , 0.018636131659 , 0.018669795245 , 0.018704019487 , 0.018737735227 , 0.018756538630 , 0.018747124821 , 0.018710901961 , 0.018667967990 , 0.018639953807 , 0.018604161218 , 0.018573077396 , 0.018550094217 , 0.018523411825 , 0.018483715132 , 0.018409028649 , 0.018296619877 , 0.018187319860 , 0.018072308972 , 0.017953632399 , 0.017857246101 , 0.017761211842 , 0.017680728808 , 0.017657628283 , 0.017648501322 , 0.017670700327 , 0.017707815394 , 0.017753398046 , 0.017824035138 , 0.017881682143 , 0.017953043804 , 0.018012998626 , 0.018057236448 , 0.018049677834 , 0.017995001748 , 0.017895046622 , 0.017729988322 , 0.017522839829 , 0.017302121967 , 0.017098423094 , 0.016947966069 , 0.016892805696 , 0.016908027232 , 0.016997382045 , 0.017163721845 , 0.017371451482 , 0.017592895776 , 0.017878061160 , 0.018199024722 , 0.018566804007 , 0.018937928602 , 0.019378701225 , 0.019932078198 , 0.020448448136 , 0.020956739783 , 0.021483207121 , 0.022014245391 , 0.022286137566 , 0.022234903648 , 0.022150812671 , 0.021970473230 , 0.021684898064 , 0.021432483569 , 0.021249620244 , 0.021129831672 , 0.021087197587 , 0.021146858111 , 0.021302009001 , 0.021511660889 , 0.021739328280 , 0.021955139935 , 0.022138863802 , 0.022286565974 , 0.022440813482 , 0.022679483518 , 0.023013198748 , 0.023392088711 , 0.023821959272 , 0.024209514260 , 0.024470470846 , 0.024595754221 , 0.024602552876 , 0.024540847167 , 0.024423368275 , 0.024308456108 , 0.024296721444 , 0.024327794090 , 0.024391032755 , 0.024445921183 , 0.024469867349 , 0.024448337033 , 0.024387339130 , 0.024298785254 , 0.024236006662 , 0.024196021259 , 0.024191856384 , 0.024229722098 , 0.024191670120 , 0.024207619950 , 0.024266710505 , 0.024365739897 , 0.024520980194 , 0.024675743654 , 0.024815469980 , 0.024875191972 , 0.024823652580 , 0.024643031880 , 0.024383738637 , 0.024085922167 , 0.023800561205 , 0.023565864190 , 0.023383857682 , 0.023225916550 , 0.023128641769 , 0.023036643863 , 0.022916525602 , 0.022808728740 , 0.022713303566 , 0.022629408166 , 0.022545622662 , 0.022463312373 , 0.022389598191 , 0.022300876677 , 0.022258242592 , 0.022305732593 , 0.022428393364 , 0.022557750344 , 0.022677697241 , 0.022715399042 , 0.022682048380 , 0.022526783869 , 0.022281991318 , 0.021965295076 , 0.021621340886 , 0.021348180249 , 0.021186970174 , 0.021121442318 , 0.021159669384 , 0.021313352510 , 0.021581992507 , 0.021940121427 , 0.022448599339 , 0.023115694523 , 0.023893579841 , 0.024689206854 , 0.025420172140 , 0.026016250253 , 0.026393806562 , 0.026498442516 , 0.026427850127 , 0.026229484007 , 0.025965338573 , 0.025745702907 , 0.025583261624 , 0.025440590456 , 0.025301575661 , 0.025074660778 , 0.024807816371 , 0.024572888389 , 0.024368325248 , 0.024259099737 , 0.024310262874 , 0.024439990520 , 0.024544628337 , 0.024622755125 , 0.024628078565 , 0.024588806555 , 0.024498449638 , 0.024356691167 , 0.024168958887 , 0.023947067559 , 0.023697892204 , 0.023452078924 , 0.023255228996 , 0.023111471906 , 0.023022204638 , 0.022961748764 , 0.022898435593 , 0.022774619982 , 0.022635726258 , 0.022519452497 , 0.022498337552 , 0.022569226101 , 0.022710964084 , 0.022841734812 , 0.022949725389 , 0.023037476465 , 0.023071371019 , 0.023076547310 , 0.023089582101 , 0.023166827857 , 0.023309929296 , 0.023476622999 , 0.023617213592 , 0.023676963523 , 0.023664494976 , 0.023603335023 , 0.023513697088 , 0.023397604004 , 0.023228360340 , 0.023031992838 , 0.022826842964 , 0.022617502138 , 0.022412799299 , 0.022233620286 , 0.022052967921 , 0.021861521527 , 0.021654566750 , 0.021456295624 , 0.021301276982 , 0.021216263995 , 0.021151691675 , 0.021091915667 , 0.021035676822 , 0.020971164107 , 0.020889876410 , 0.020783627406 , 0.020672328770 , 0.020573602989 , 0.020517220721 , 0.020507300273 , 0.020547488704 , 0.020567199215 , 0.020588474348 , 0.020559499040 , 0.020538944751 , 0.020542344078 , 0.020576838404 , 0.020625879988 , 0.020686324686 , 0.020757233724 , 0.020809506997 , 0.020821997896 , 0.020798692480 , 0.020776683465 , 0.020763436332 , 0.020732050762 , 0.020712634549 , 0.020746266469 , 0.020846813917 , 0.021012833342 , 0.021196626127 , 0.021434249356 , 0.021667936817 , 0.021900856867 , 0.022108942270 , 0.022291095927 , 0.022459669039 , 0.022576116025 , 0.022609284148 , 0.022598134354 , 0.022569654509 , 0.022522984073 , 0.022425385192 , 0.022329906002 , 0.022295659408 , 0.022351123393 , 0.022472480312 , 0.022584967315 , 0.022694906220 , 0.022784339264 , 0.022876525298 , 0.022971162573 , 0.023083465174 , 0.023187389597 , 0.023267239332 , 0.023315539584 , 0.023286813870 , 0.023175278679 , 0.023052969947 , 0.022951310501 , 0.022887134925 , 0.022831901908 , 0.022777261212 , 0.022715963423 , 0.022654945031 , 0.022609978914 , 0.022571707144 , 0.022488122806 , 0.022378539667 , 0.022241881117 , 0.022099925205 , 0.022017268464 , 0.021957205608 , 0.021942881867 , 0.021949941292 , 0.021981427446 , 0.022045001388 , 0.022142695263 , 0.022188393399 , 0.022203775123 , 0.022217988968 , 0.022237153724 , 0.022255139425 , 0.022298345342 , 0.022312344983 , 0.022329568863 , 0.022361664101 , 0.022463316098 , 0.022641753778 , 0.022843174636 , 0.023015357554 , 0.023199414834 , 0.023349156603 , 0.023492088541 , 0.023605383933 , 0.023677842692 , 0.023720519617 , 0.023728946224 , 0.023717308417 , 0.023696847260 , 0.023672454059 , 0.023643994704 , 0.023669324815 , 0.023743810132 , 0.023835705593 , 0.023898741230 , 0.023939339444 , 0.023980630562 , 0.023999357596 , 0.023959474638 , 0.023860841990 , 0.023723199964 , 0.023550437763 , 0.023372689262 , 0.023211045191 , 0.023099148646 , 0.022997012362 , 0.022901533172 , 0.022827090696 , 0.022720761597 , 0.022574702278 , 0.022423570976 , 0.022310873494 , 0.022220812738 , 0.022146299481 , 0.022113053128 , 0.022121386603 , 0.022201031446 , 0.022292876616 , 0.022352799773 , 0.022374501452 , 0.022354291752 , 0.022292988375 , 0.022219875827 , 0.022150890902 , 0.022080326453 , 0.022054359317 , 0.022060731426 , 0.022079931572 , 0.022109704092 , 0.022119916975 , 0.022127680480 , 0.022090965882 , 0.022084278986 , 0.022174542770 , 0.022340884432 , 0.022550471127 , 0.022752173245 , 0.022959420457 , 0.023107232526 , 0.023161912337 , 0.023134471849 , 0.023054331541 , 0.022911788896 , 0.022762408480 , 0.022628007457 , 0.022512795404 , 0.022382484749 , 0.022247105837 , 0.022138038650 , 0.022061703727 , 0.022049868479 , 0.022112585604 , 0.022227445617 , 0.022399699315 , 0.022576892748 , 0.022718019783 , 0.022791244090 , 0.022810781375 , 0.022783672437 , 0.022723577917 , 0.022666318342 , 0.022646313533 , 0.022687884048 , 0.022831790149 , 0.022928161547 , 0.023051954806 , 0.023172469810 , 0.023232430220 , 0.023266615346 , 0.023283936083 , 0.023267932236 ]
mse_test_53_pred45 =  [ 0.000359110069 , 0.000154024558 , 0.000096203243 , 0.000072186769 , 0.000063490712 , 0.000056827863 , 0.000061661740 , 0.000060576134 , 0.000050331579 , 0.000044015254 , 0.000041010335 , 0.000042345240 , 0.000044878325 , 0.000051869025 , 0.000057337107 , 0.000069231901 , 0.000074462994 , 0.000088380628 , 0.000088779467 , 0.000084177787 , 0.000073987161 , 0.000076569944 , 0.000081053870 , 0.000081509002 , 0.000080848353 , 0.000084760897 , 0.000091845694 , 0.000087223714 , 0.000081044964 , 0.000078137411 , 0.000069936774 , 0.000068551431 , 0.000067031615 , 0.000062776737 , 0.000058035894 , 0.000059734270 , 0.000064798907 , 0.000085147185 , 0.000109754095 , 0.000125619772 , 0.000134662012 , 0.000138993739 , 0.000148887324 , 0.000154474124 , 0.008840350434 , 0.005281588528 , 0.000132002402 , 0.000120686920 , 0.000111362519 , 0.000100638274 , 0.000352488249 , 0.000703627185 , 0.001114683575 , 0.001568096573 , 0.002051628428 , 0.002573359758 , 0.003100687871 , 0.003557776799 , 0.003861583536 , 0.004050483461 , 0.004209797829 , 0.004358443897 , 0.004507587757 , 0.004637899809 , 0.004741582554 , 0.004797249567 , 0.004798164126 , 0.004732517991 , 0.004633054603 , 0.004469773266 , 0.004289909266 , 0.004112898372 , 0.003954747692 , 0.003828096902 , 0.003721301677 , 0.003621614305 , 0.003521061735 , 0.003389235819 , 0.003270091489 , 0.003156631486 , 0.003093589330 , 0.003103947500 , 0.003205033951 , 0.003378917230 , 0.003595228074 , 0.003820751561 , 0.004013282713 , 0.004160774872 , 0.004263364244 , 0.004337783437 , 0.004399827216 , 0.004456149414 , 0.004518297501 , 0.004595463164 , 0.004698150326 , 0.004848436452 , 0.005023323465 , 0.005176814273 , 0.005277010147 , 0.005307663698 , 0.005286172032 , 0.005229196046 , 0.005152096506 , 0.005070711020 , 0.005001627374 , 0.004973777104 , 0.004996369127 , 0.005065323785 , 0.005165170413 , 0.005290169269 , 0.005430993158 , 0.005597848911 , 0.005785036832 , 0.005995551124 , 0.006229665596 , 0.006461567711 , 0.006699508056 , 0.006941996980 , 0.007160807494 , 0.007325559855 , 0.007419196889 , 0.007415384054 , 0.007346658036 , 0.007221361622 , 0.007029671222 , 0.006805350538 , 0.006566251162 , 0.006346283946 , 0.006194165442 , 0.006140836049 , 0.006200704724 , 0.006319034845 , 0.006440363359 , 0.006492208224 , 0.006424010266 , 0.006304865237 , 0.006145575549 , 0.005947831552 , 0.005703206640 , 0.005415472668 , 0.005122072063 , 0.004906273913 , 0.004782666918 , 0.004718154669 , 0.004703027196 , 0.004728381522 , 0.004773070570 , 0.004828133620 , 0.004891809076 , 0.004963843152 , 0.005061924923 , 0.005208635703 , 0.005379131064 , 0.005509776529 , 0.005574815441 , 0.005548426881 , 0.005498873536 , 0.005449078511 , 0.005408224184 , 0.005387122277 , 0.005390198436 , 0.005433844402 , 0.005534470081 , 0.005685938057 , 0.005880464334 , 0.006072574761 , 0.006246715784 , 0.006358176004 , 0.006425729021 , 0.006415322889 , 0.006391018163 , 0.006319715176 , 0.006204014178 , 0.006099173799 , 0.006006047595 , 0.005970406812 , 0.005934720393 , 0.005845943000 , 0.005717351567 , 0.005576367956 , 0.005446476862 , 0.005338482559 , 0.005291134585 , 0.005338955205 , 0.005484736059 , 0.005708659068 , 0.006027721334 , 0.006372064352 , 0.006720739882 , 0.007021720987 , 0.007278370205 , 0.007469356060 , 0.007597393822 , 0.007675621659 , 0.007700913120 , 0.007723627146 , 0.007747727912 , 0.007775506005 , 0.007795006037 , 0.007815822028 , 0.007865403779 , 0.007930105552 , 0.007995733060 , 0.008046668023 , 0.008110751398 , 0.008270030841 , 0.008584679104 , 0.009011483751 , 0.009489978664 , 0.010009802878 , 0.010467370041 , 0.010810387321 , 0.011002979241 , 0.011078566313 , 0.011087187566 , 0.011079568416 , 0.011051603593 , 0.010934009217 , 0.010685888119 , 0.010264305398 , 0.009699127637 , 0.009036476724 , 0.008353820071 , 0.007765963208 , 0.007309643086 , 0.007015992422 , 0.006865851115 , 0.006857734174 , 0.006915301550 , 0.007047936786 , 0.007132879924 , 0.007178359199 , 0.007213116158 , 0.007247539237 , 0.007318309974 , 0.007425158750 , 0.007554601412 , 0.007680386771 , 0.007792509627 , 0.007902590558 , 0.008009746671 , 0.008093138225 , 0.008149860427 , 0.008168672211 , 0.008131188340 , 0.008009011857 , 0.007828519680 , 0.007625696715 , 0.007404649630 , 0.007181372959 , 0.006968278904 , 0.006787271705 , 0.006640714128 , 0.006529094651 , 0.006465648767 , 0.006422809791 , 0.006394123659 , 0.006368733943 , 0.006353455130 , 0.006321507972 , 0.006284750998 , 0.006242133211 , 0.006205401849 , 0.006175460294 , 0.006146530155 , 0.006133840885 , 0.006140965968 , 0.006185660604 , 0.006242036354 , 0.006285638083 , 0.006296825130 , 0.006242095958 , 0.006127856206 , 0.005987093318 , 0.005830347072 , 0.005686677527 , 0.005556988996 , 0.005454434082 , 0.005397937726 , 0.005372760817 , 0.005383661017 , 0.005422640126 , 0.005461000372 , 0.005477359053 , 0.005462469999 , 0.005436758045 , 0.005393553525 , 0.005352659151 , 0.005372976884 , 0.005444737151 , 0.005525096785 , 0.005566681270 , 0.005606986582 , 0.005627230275 , 0.005646324251 , 0.005688101053 , 0.005778945517 , 0.005926162470 , 0.006104278844 , 0.006276377011 , 0.006436177995 , 0.006580967922 , 0.006695387885 , 0.006796624511 , 0.006874810904 , 0.006930455565 , 0.006985004991 , 0.007084269542 , 0.007219376042 , 0.007376085967 , 0.007534319535 , 0.007734817918 , 0.007946926169 , 0.008145050146 , 0.008301703259 , 0.008389760740 , 0.008381739259 , 0.008297962137 , 0.008125972003 , 0.007919578813 , 0.007689770311 , 0.007532453630 , 0.007573727518 , 0.007783707231 , 0.008114891127 , 0.008523569442 , 0.008964330889 , 0.009394028224 , 0.009817114100 , 0.010194883682 , 0.010467275046 , 0.010653597303 , 0.010791632347 , 0.010905150324 , 0.011017148383 , 0.011119035073 , 0.011206235737 , 0.011323257349 , 0.011415447108 , 0.011478380300 , 0.011453256942 , 0.011330951937 , 0.011122505181 , 0.010849103332 , 0.010560861789 , 0.010301304981 , 0.010036022402 , 0.009726868011 , 0.009372830391 , 0.008980996907 , 0.008601258509 , 0.008280062117 , 0.008039896376 , 0.007889854722 , 0.007822853513 , 0.007806396578 , 0.007745195646 , 0.007629991043 , 0.007454693783 , 0.007276745513 , 0.007092026528 , 0.006921103224 , 0.006825260818 , 0.006831547711 , 0.006924471352 , 0.007127820048 , 0.007437504362 , 0.007830760442 , 0.008299272507 , 0.008806485683 , 0.009227753617 , 0.009423534386 , 0.009286220185 , 0.008984678425 , 0.008599179797 , 0.008187444881 , 0.007813301869 , 0.007481573615 , 0.007178755011 , 0.006884045433 , 0.006615320686 , 0.006376673933 , 0.006210147869 , 0.006124335807 , 0.006109623238 , 0.006147548556 , 0.006201252807 , 0.006240470335 , 0.006267184392 , 0.006254807115 , 0.006180457771 , 0.006071841810 , 0.005978370085 , 0.005937319715 , 0.005948722828 , 0.006004652008 , 0.006035721395 , 0.005961785093 , 0.005840798374 , 0.005693104584 , 0.005509512033 , 0.005293221679 , 0.005086779594 , 0.004901871551 , 0.004751603585 , 0.004649013747 , 0.004568795208 , 0.004514767788 , 0.004484231584 , 0.004502498079 , 0.004564740229 , 0.004666144960 , 0.004824211821 , 0.005062999669 , 0.005337138195 , 0.005590121727 , 0.005790370051 , 0.005900130142 , 0.005846658256 , 0.005688516423 , 0.005424140021 , 0.005094384309 , 0.004738695454 , 0.004402562976 , 0.004157833289 , 0.003992616665 , 0.003919957671 , 0.003929967526 , 0.004003255162 , 0.004132700153 , 0.004302779213 , 0.004487639759 , 0.004632580560 , 0.004724754021 , 0.004782288801 , 0.004835517146 , 0.004894507583 , 0.004952481482 , 0.005037979688 , 0.005199532025 , 0.005410847720 , 0.005616847426 , 0.005758419167 , 0.005857542623 , 0.005911861081 , 0.005927493796 , 0.005921501666 , 0.005864809733 , 0.005749918055 , 0.005582389887 , 0.005373418331 , 0.005140796304 , 0.004913837183 , 0.004720908124 , 0.004563035909 , 0.004432138987 , 0.004335229751 , 0.004275851417 , 0.004266773816 , 0.004293320235 , 0.004339589272 , 0.004449838307 , 0.004595178179 , 0.004773319699 , 0.004980200436 , 0.005218788981 , 0.005381256342 , 0.005455793347 , 0.005497917533 , 0.005522025283 , 0.005569564644 , 0.005633682013 , 0.005648035090 , 0.005543373059 , 0.005415744614 , 0.005362367723 , 0.005382238422 , 0.005485972855 , 0.005660997238 , 0.005895293783 , 0.006123601925 , 0.006356211379 , 0.006533951964 , 0.006649035495 , 0.006699488033 , 0.006682473700 , 0.006630655844 , 0.006575197447 , 0.006532459054 , 0.006505709141 , 0.006447086111 , 0.006369663402 , 0.006322137546 , 0.006293246523 , 0.006244435906 , 0.006218984723 , 0.006229871884 , 0.006241262425 , 0.006239104550 , 0.006225040648 , 0.006180347409 , 0.006096966565 , 0.005973519757 , 0.005861661863 , 0.005760975648 , 0.005694894586 , 0.005623202305 , 0.005539168138 , 0.005477504805 , 0.005431738216 , 0.005381814670 , 0.005319654476 , 0.005248918664 , 0.005191235337 , 0.005162879359 , 0.005179511849 , 0.005221808329 , 0.005366791040 , 0.005551159382 , 0.005788832903 , 0.006122522056 , 0.006518601906 , 0.006891192868 , 0.007212033495 , 0.007446563337 , 0.007590773981 , 0.007645253092 , 0.007611535024 , 0.007544416934 , 0.007501512766 , 0.007485876326 , 0.007512152661 , 0.007558281999 , 0.007539558690 , 0.007403083611 , 0.007189435419 , 0.007013726514 , 0.006926559377 , 0.006912821904 , 0.006988092791 , 0.007080715150 , 0.007182836998 , 0.007327293977 , 0.007478544954 , 0.007609376684 , 0.007619470824 , 0.007447209675 , 0.007185868919 , 0.006890018936 , 0.006611615885 , 0.006388848647 , 0.006283361465 , 0.006289701443 , 0.006382731721 , 0.006479092408 , 0.006567705888 , 0.006651020143 , 0.006693408359 , 0.006702336017 , 0.006748349871 , 0.006842006464 , 0.006976110861 , 0.007164088544 , 0.007359584328 , 0.007547507063 , 0.007685186807 , 0.007751313969 , 0.007737575099 , 0.007606558502 , 0.007423145697 , 0.007228900213 , 0.007063027937 , 0.006943428423 , 0.006882065907 , 0.006884246599 , 0.006926270667 , 0.006973059382 , 0.007039857563 , 0.007109349128 , 0.007186932024 , 0.007250627968 , 0.007273838390 , 0.007248849142 , 0.007170155179 , 0.007006226573 , 0.006795534398 , 0.006560483947 , 0.006340891123 , 0.006136136595 , 0.005980674643 , 0.005860915873 , 0.005793940276 , 0.005753800739 , 0.005763574969 , 0.005811670795 , 0.005894508678 , 0.005982104223 , 0.006089311093 , 0.006174967159 , 0.006192905828 , 0.006104224827 , 0.005963254720 , 0.005792852491 , 0.005645494908 , 0.005554183852 , 0.005515284371 , 0.005518438760 , 0.005547463428 , 0.005600837991 , 0.005654785782 , 0.005693753716 , 0.005704434123 , 0.005666879471 , 0.005606389139 , 0.005530600902 , 0.005457613617 , 0.005379023030 , 0.005307945888 , 0.005240792409 , 0.005178840365 , 0.005117397290 , 0.005038149189 , 0.004945070948 , 0.004845802672 , 0.004749523941 , 0.004662922118 , 0.004578681197 , 0.004469680600 , 0.004353736993 , 0.004246296827 , 0.004157019313 , 0.004098266363 , 0.004060886335 , 0.004045213107 , 0.004044548143 , 0.004050695803 , 0.004064552486 , 0.004071053583 , 0.004056680482 , 0.004003878217 , 0.003914611414 , 0.003806256922 , 0.003682663897 , 0.003564372426 , 0.003465128364 , 0.003390647471 , 0.003365195356 , 0.003405482741 , 0.003496129299 , 0.003638228402 , 0.003811612958 , 0.004007911775 , 0.004213770386 , 0.004443976562 , 0.004677734803 , 0.004929192364 , 0.005174166523 , 0.005462165456 , 0.005799481645 , 0.006081968080 , 0.006360161584 , 0.006668318529 , 0.007019303273 , 0.007182034198 , 0.007042966783 , 0.006891708821 , 0.006656750571 , 0.006314016879 , 0.006012347061 , 0.005796283949 , 0.005656171124 , 0.005594092887 , 0.005618864670 , 0.005726845935 , 0.005907088518 , 0.006108752917 , 0.006297559012 , 0.006447253283 , 0.006558542605 , 0.006652929354 , 0.006772068795 , 0.006950832903 , 0.007152079139 , 0.007384534925 , 0.007625793573 , 0.007806494832 , 0.007907089777 , 0.007907873951 , 0.007863913663 , 0.007779281586 , 0.007695409004 , 0.007686455268 , 0.007669281680 , 0.007600899786 , 0.007463118527 , 0.007261151914 , 0.006992334966 , 0.006661690772 , 0.006325580645 , 0.006036046892 , 0.005825135857 , 0.005678687245 , 0.005573712755 , 0.005442991853 , 0.005409153644 , 0.005445972551 , 0.005535454024 , 0.005657495465 , 0.005770204589 , 0.005847355817 , 0.005857404787 , 0.005780543666 , 0.005632768851 , 0.005449214485 , 0.005292793736 , 0.005212411284 , 0.005221284926 , 0.005313793197 , 0.005471674725 , 0.005664633587 , 0.005839383695 , 0.005952866282 , 0.006019625347 , 0.006060180720 , 0.006078346167 , 0.006072400603 , 0.006032332778 , 0.005985767115 , 0.005957953166 , 0.005990903359 , 0.006079943385 , 0.006229016930 , 0.006371306721 , 0.006491139531 , 0.006574472878 , 0.006613457110 , 0.006570644211 , 0.006475774571 , 0.006303317379 , 0.006085820962 , 0.005882943515 , 0.005742853507 , 0.005659422372 , 0.005665607285 , 0.005792726297 , 0.006047289819 , 0.006426345557 , 0.006945176050 , 0.007585719693 , 0.008287656121 , 0.008998290636 , 0.009661792777 , 0.010221314617 , 0.010638657957 , 0.010831999592 , 0.010852406733 , 0.010710555129 , 0.010421321727 , 0.010108384304 , 0.009826052003 , 0.009559066035 , 0.009309479035 , 0.009048458189 , 0.008781814016 , 0.008536624722 , 0.008312438615 , 0.008169868030 , 0.008159013465 , 0.008227332495 , 0.008286262862 , 0.008303251117 , 0.008271452039 , 0.008191258647 , 0.008061398752 , 0.007888517343 , 0.007713973988 , 0.007544567343 , 0.007387996186 , 0.007250025868 , 0.007147261407 , 0.007069890853 , 0.007001334336 , 0.006947439164 , 0.006881724577 , 0.006815668661 , 0.006765164901 , 0.006757265422 , 0.006798277143 , 0.006876736879 , 0.006973863114 , 0.007050184067 , 0.007074826863 , 0.007057704497 , 0.006987832021 , 0.006896656472 , 0.006825853605 , 0.006803123746 , 0.006826913450 , 0.006860539317 , 0.006827419158 , 0.006692424417 , 0.006479812786 , 0.006229720544 , 0.005968701094 , 0.005729213823 , 0.005505135283 , 0.005286812317 , 0.005088204984 , 0.004934540950 , 0.004848745186 , 0.004840272479 , 0.004888999276 , 0.004959026817 , 0.005052618682 , 0.005158001557 , 0.005314929411 , 0.005526599940 , 0.005724638700 , 0.005863040220 , 0.005947449710 , 0.005985276308 , 0.005977179855 , 0.005944668781 , 0.005907357205 , 0.005878219847 , 0.005879456643 , 0.005916444119 , 0.005982525647 , 0.006032277364 , 0.006073212251 , 0.006103443448 , 0.006140208337 , 0.006181530189 , 0.006239937153 , 0.006326230709 , 0.006415573414 , 0.006478611380 , 0.006493691355 , 0.006475070957 , 0.006405774970 , 0.006335205864 , 0.006284289062 , 0.006277054548 , 0.006325822789 , 0.006405273452 , 0.006485908758 , 0.006559740286 , 0.006573342253 , 0.006542820483 , 0.006428574678 , 0.006281178910 , 0.006134103518 , 0.006008114666 , 0.005912678782 , 0.005822050851 , 0.005670287181 , 0.005485120695 , 0.005290954839 , 0.005094218533 , 0.004884668160 , 0.004726285581 , 0.004672584590 , 0.004733557813 , 0.004852693994 , 0.004944378976 , 0.004984179046 , 0.004961821716 , 0.004911355209 , 0.004867848475 , 0.004860375542 , 0.004856426734 , 0.004839124624 , 0.004798784386 , 0.004693204071 , 0.004526196979 , 0.004370431416 , 0.004276729655 , 0.004242708441 , 0.004279594403 , 0.004368007649 , 0.004458647687 , 0.004541811999 , 0.004626778420 , 0.004686014261 , 0.004681751598 , 0.004621930886 , 0.004516724963 , 0.004435264971 , 0.004420659505 , 0.004408816341 , 0.004405335523 , 0.004371747840 , 0.004331966396 , 0.004305592272 , 0.004291001242 , 0.004231854808 , 0.004160854500 , 0.004118616227 , 0.004124647472 , 0.004167854320 , 0.004245918244 , 0.004343815614 , 0.004467783496 , 0.004626709502 , 0.004873854574 , 0.005219042767 , 0.005615855101 , 0.005997061264 , 0.006368066650 , 0.006682818290 , 0.006960975472 , 0.007165144663 , 0.007287292276 , 0.007345325779 , 0.007348379586 , 0.007301043253 , 0.007199453190 , 0.007075234782 , 0.006959276274 , 0.006879925728 , 0.006838263478 , 0.006824302953 , 0.006789410952 , 0.006713869050 , 0.006614944432 , 0.006505436730 , 0.006384164561 , 0.006243893411 , 0.006068024319 , 0.005873954389 , 0.005690800492 , 0.005575110670 , 0.005557283293 , 0.005611936096 , 0.005704214796 , 0.005803358275 , 0.005821789149 , 0.005738277454 , 0.005617676768 , 0.005481636617 , 0.005358783528 , 0.005275201518 , 0.005237909500 , 0.005252317991 , 0.005315435585 , 0.005387300160 , 0.005412529688 , 0.005384847987 , 0.005305018742 , 0.005181143060 , 0.005036309361 , 0.004883588292 , 0.004735879600 , 0.004609426484 , 0.004501045216 , 0.004410736728 , 0.004329216667 , 0.004259922076 , 0.004212406464 , 0.004173837602 , 0.004189870786 , 0.004292041529 , 0.004462593235 , 0.004658044782 , 0.004832552280 , 0.005011322442 , 0.005109976977 , 0.005108835641 , 0.005018433556 , 0.004859283566 , 0.004662697669 , 0.004481960554 , 0.004332681652 , 0.004201408941 , 0.004061013926 , 0.003920526244 , 0.003779582679 , 0.003657812253 , 0.003585029626 , 0.003572060959 , 0.003613007255 , 0.003706475021 , 0.003832905320 , 0.003958755173 , 0.004053020850 , 0.004125474021 , 0.004193053115 , 0.004264215939 , 0.004354785662 , 0.004476887174 , 0.004640945699 , 0.004876397084 , 0.005080940668 , 0.005282350350 , 0.005490538199 , 0.005668475758 , 0.005816495512 , 0.005956364330 , 0.006083528046 ]
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(np.arange(len(mse_test_31_pred10)), mse_test_31_pred10, label="31 Joints (pred_steps=10)")
ax.plot(np.arange(len(mse_test_53_pred0)), mse_test_53_pred0, label="53 Joints (pred_steps=0)")
ax.plot(np.arange(len(mse_test_53_pred10)), mse_test_53_pred10, label="53 Joints (pred_steps=10)")
ax.plot(np.arange(len(mse_test_53_pred45)), mse_test_53_pred45, label="53 Joints (pred_steps=45)")
# ax.plot(np.linspace(0,len(mse_test)*1000,20), mse_test, label="All Joints")
ax.set_xlabel("Timesteps in the Future", fontsize=16)
ax.set_ylabel("Mean-Squared Error", fontsize=16)
ax.legend(fontsize=16)
ax.tick_params(labelsize=14)
ax.set_xlim(0,100)
ax.grid(True, linestyle='-.')
ax.set_title("Test MSE", fontsize=24)
plt.savefig("mse_test_first100.png")